In [14]:
from google.cloud import storage
from zipfile import ZipFile
from zipfile import is_zipfile
import io

def zipextract(bucketname, zipfilename_with_path):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketname)

    destination_blob_pathname = zipfilename_with_path
        
    blob = bucket.blob(destination_blob_pathname)
    zipbytes = io.BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                blob = bucket.blob(zipfilename_with_path + "/" + contentfilename)
                blob.upload_from_string(contentfile)

#zipextract("different-images", "archive.zip") # if the file is gs://different-images/archive.zip







In [77]:
from google.cloud import storage
import pandas as pd

f = open("data_init.txt", "w")
f.write("path" + "," + "label" + "\n")

data_store_in_list = []
def file_location_and_Labelling(bucket_name, prefixname):
    bucketname = bucket_name #'different-images'
    client = storage.Client()

    #for blob in client.list_blobs(bucketname, prefix='archive.zip'):
    #  print(str(blob))
    allFiles=[]
    for blob in client.list_blobs(bucketname, prefix=prefixname): #'archive.zip/seg_train/seg_train'):
      #print(blob.name)
        allFiles.append(blob.name)

    #print(len(allFiles))
    for pathFile in allFiles:   
        if pathFile.split("/")[3] in ('buildings', 'forest', 'mountain', 'sea'):
            #print(pathFile, pathFile.split("/")[3])
            f.write("gs://different-images/"+pathFile + "," + pathFile.split("/")[3])
            f.write("\n")
            
            #data_store_in_list.append(pathFile pathFile.split("/")[3])
    
## Output --> archive.zip/seg_train/seg_train/buildings/0.jpg buildings
Bucket = 'different-images'
Prefix1 = 'archive.zip/seg_train/seg_train'
file_location_and_Labelling(Bucket, Prefix1)
#print(len(data_store_in_list))
f.close()


read_file = pd.read_csv ('data_init.txt')
read_file.to_csv ('train_data.csv', index=False, header= False)


In [78]:
!gsutil cp train_data.csv gs://different-images/archive.zip/seg_train/seg_train/

Copying file://train_data.csv [Content-Type=text/csv]...
/ [1 files][708.8 KiB/708.8 KiB]                                                
Operation completed over 1 objects/708.8 KiB.                                    


In [87]:
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 20.1 MB/s eta 0:00:0000:0100:01


In [31]:
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
print('')
!which ffmpeg
print('Done!')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.0M  100 39.0M    0     0  29.3M      0  0:00:01  0:00:01 --:--:-- 29.3M
env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:./ffmpeg-5.1.1-amd64-static

./ffmpeg-5.1.1-amd64-static/ffmpeg
Done!


In [34]:
# !pip install pydub

# !pip install av

#apt-get install ffmpeg

#brew install ffmpeg


from pydub import AudioSegment
import random

from moviepy.editor import AudioFileClip, ImageClip


def get_random_10sec_audio_clip(srcAudioClip, destAudioClip):
    sound = AudioSegment.from_mp3(srcAudioClip)

    music_len = int(len(sound)) -20000
    irand = random.randint(0, music_len)
    
    # Concatenation is just adding
    music = sound[irand:irand+10000]

    # writing mp3 files is a one liner
    music.export(destAudioClip, format="mp3")


def add_static_image_to_audio(musicSrc, imageSrc, videoSrc):
    
    audio_clip = AudioFileClip(musicSrc)
    image_clip = ImageClip(imageSrc)
    #image_clip = ImageClip("/home/jupyter/gcp-demo/55.png")
    
    video_clip = image_clip.set_audio(audio_clip)
    # specify the duration of the new clip to be the duration of the audio clip
    video_clip.duration = 10
    # set the FPS to 1
    video_clip.fps = 1
    # write the resuling video clip
    video_clip.write_videofile(videoSrc)


music_src = "/home/jupyter/gcp-demo/MountainMusic.mp3"
music_dest = "/home/jupyter/gcp-demo/10secMusic.mp3"

get_random_10sec_audio_clip(music_src, music_dest)

singleImageSrc = "/home/jupyter/gcp-demo/42.jpg"
videoSrc = "/home/jupyter/gcp-demo/10secVideo.ogv"

add_static_image_to_audio(music_dest, singleImageSrc, videoSrc)

Moviepy - Building video /home/jupyter/gcp-demo/10secVideo.ogv.
MoviePy - Writing audio in 10secVideoTEMP_MPY_wvf_snd.ogg


MoviePy - Done.
Moviepy - Writing video /home/jupyter/gcp-demo/10secVideo.ogv



Moviepy - Done !
Moviepy - video ready /home/jupyter/gcp-demo/10secVideo.ogv


In [35]:
from google.cloud import aiplatform


In [7]:
import base64

from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict


def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    with open(filename, "rb") as f:
        file_content = f.read()

    # The format of each instance should conform to the deployed model's prediction input schema.
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]
    # See gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml for the format of the parameters.
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5, max_predictions=5,
    ).to_value()
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/image_classification_1.0.0.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))
        
    return predictions
        
predictions_in_main_function = predict_image_classification_sample(
    project="643948302050",
    endpoint_id="8669152205758005248",
    location="us-central1",
    filename="/home/jupyter/gcp-demo/42.jpg"
)

for prediction in predictions_in_main_function:
    print(" prediction:", dict(prediction))

print(prediction)
print(prediction['displayNames'])
print(prediction['displayNames'][0])





response
 deployed_model_id: 2886864535749132288
 prediction: {'ids': ['2776353171797180416'], 'confidences': [0.858314157], 'displayNames': ['mountain']}
 prediction: {'ids': ['2776353171797180416'], 'confidences': [0.858314157], 'displayNames': ['mountain']}
['mountain']
mountain


In [33]:
from pydub import AudioSegment
import random

from moviepy.editor import AudioFileClip, ImageClip

import base64

from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict

from datetime import date

def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    with open(filename, "rb") as f:
        file_content = f.read()

    # The format of each instance should conform to the deployed model's prediction input schema.
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]
    # See gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml for the format of the parameters.
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5, max_predictions=5,
    ).to_value()
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    #print("response")
    #print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/image_classification_1.0.0.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))
        
    return predictions


def get_random_10sec_audio_clip(srcAudioClip, destAudioClip):
    sound = AudioSegment.from_mp3(srcAudioClip)

    music_len = int(len(sound)) -20000
    irand = random.randint(0, music_len)
    
    # Concatenation is just adding
    music = sound[irand:irand+10000]

    # writing mp3 files is a one liner
    music.export(destAudioClip, format="mp3")


def add_static_image_to_audio(musicSrc, imageSrc, videoSrc):
    
    audio_clip = AudioFileClip(musicSrc)
    image_clip = ImageClip(imageSrc)
    #image_clip = ImageClip("/home/jupyter/gcp-demo/55.png")
    
    video_clip = image_clip.set_audio(audio_clip)
    # specify the duration of the new clip to be the duration of the audio clip
    video_clip.duration = 10
    # set the FPS to 1
    video_clip.fps = 1
    # write the resuling video clip
    video_clip.write_videofile(videoSrc)
    
    print("--------------------------------------")
    print("-------------File Created-------------")
    print("--------------------------------------")




####################################

#######  Here is the Uput ##########
singleImageSrc="/home/jupyter/gcp-demo/AS-sea.jpeg"

####################################


predictions_in_main_function = predict_image_classification_sample(
    project="PROJECT_ID",
    endpoint_id="ENDPOINT_ID",
    location="us-central1",
    filename = singleImageSrc
)

for prediction_main in predictions_in_main_function:
    print(" prediction:", dict(prediction_main))

#print(prediction_main['displayNames'][0])

music_src = ""
music_dest = ""
videoSrc = ""

if prediction['displayNames'][0] == "mountain":
    music_src = "/home/jupyter/gcp-demo/MountainMusic.mp3"
    music_dest = "/home/jupyter/gcp-demo/10secMusic.mp3"       
    videoSrc = "/home/jupyter/gcp-demo/10secVideo-%s%s.ogv" %(prediction_main['displayNames'][0], date.today().strftime("%Y%b%d"))

elif prediction['displayNames'][0] == "sea":
    music_src = "/home/jupyter/gcp-demo/SeaMusic.mp3"
    music_dest = "/home/jupyter/gcp-demo/10secMusic.mp3"       
    videoSrc = "/home/jupyter/gcp-demo/10secVideo-%s%s.ogv" %(prediction_main['displayNames'][0], date.today().strftime("%Y%b%d"))
    

get_random_10sec_audio_clip(music_src, music_dest)
add_static_image_to_audio(music_dest, singleImageSrc, videoSrc)


response
 deployed_model_id: 2886864535749132288
 prediction: {'confidences': [0.977521181], 'displayNames': ['sea'], 'ids': ['1623431667190333440']}
 prediction: {'confidences': [0.977521181], 'displayNames': ['sea'], 'ids': ['1623431667190333440']}
sea
/home/jupyter/gcp-demo/SeaMusic.mp3
/home/jupyter/gcp-demo/10secMusic.mp3
/home/jupyter/gcp-demo/10secVideo-sea2022Oct08.ogv
Moviepy - Building video /home/jupyter/gcp-demo/10secVideo-sea2022Oct08.ogv.
MoviePy - Writing audio in 10secVideo-sea2022Oct08TEMP_MPY_wvf_snd.ogg


MoviePy - Done.
Moviepy - Writing video /home/jupyter/gcp-demo/10secVideo-sea2022Oct08.ogv



Moviepy - Done !
Moviepy - video ready /home/jupyter/gcp-demo/10secVideo-sea2022Oct08.ogv
--------------------------------------
-------------File Created-------------
--------------------------------------
